In [5]:
import numpy as np
import pandas as pd
import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial

data = pd.read_csv('/Users/maniroop/Downloads/kmeans_data/data.csv')
labels = pd.read_csv('/Users/maniroop/Downloads/kmeans_data/label.csv',names=['label'],header=None)

data.head()

data.count()

#Data splitting step
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split( data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( labels, test_size=0.08, random_state=42)

class KMeans:
    
    def SSE_calculation(self, centroid_value_dict, centroid_dict,data):
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0
            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a,b in zip(centroid_value_dict[i],dp):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def Centroids_initializing(self,data,K):
        m = data.shape[0]
        centroid_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
    
    def jaccard_similarity(self,centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def Kmeans_train(self,data,K,max_iter=20,mode=1,tol=10):
        centroid_value_dict = self.Centroids_initializing(data,K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccard_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value_dict)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
            
            
            current_tol=-1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                current_tol = max(change, current_tol)
                
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centroid_value_dict,centroid_dict

def prediction_labels(C, S, labels):
    '''
    Input : C -> Centroids
            S -> Set of Indicies corresponding to Centroid C
            data -> Data used to form clusters
    Output : Returns an array of size K having labels based on majority voting in the cluster
    '''
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

def jaccard_similarity(centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

model1 = KMeans()
centroids1,clusters1 = model1.Kmeans_train(data,10, max_iter=100,mode=1)

Euclidean_SSE = model1.SSE_calculation(centroids1,clusters1,data)

print("Euclidean SSE:",Euclidean_SSE)

cluster_labels1 = prediction_labels(centroids1,clusters1,labels)
cluster_labels1

Accuracy_Euclidean = accuracy(centroids1, cluster_labels1,test_data,test_labels)
Accuracy_Euclidean

model2 = KMeans()
centroids2,clusters2 = model2.Kmeans_train(data,10, max_iter=100,mode=2)
Jaccard_SSE = model2.SSE_calculation(centroids2,clusters2,data)

print("Jacard SSE:",Jaccard_SSE)

cluster_labels2 = prediction_labels(centroids2,clusters2,labels)
cluster_labels2

Accuracy_Jaccard = accuracy(centroids2, cluster_labels2,test_data,test_labels)
Accuracy_Jaccard

model3 = KMeans()
centroids3,clusters3 = model3.Kmeans_train(data,10, max_iter = 100,mode=3)

Cosine_SSE = model3.SSE_calculation(centroids3,clusters3,data)

print("SSE of Euclidean-K-means:",Euclidean_SSE)
print("SSE of Cosine-K-means:",Jaccard_SSE)
print("SSE of Jacards-K-means:",Cosine_SSE)

cluster_labels3 = prediction_labels(centroids3,clusters3,labels)
cluster_labels3

Accuracy_Cosine = accuracy(centroids3, cluster_labels3,test_data,test_labels)
Accuracy_Euclidean
Accuracy_Jaccard
Accuracy_Cosine

print("Accuracy of Jacards-K-means:",Accuracy_Euclidean)
print("Accuracy of Euclidean-K-means:",Accuracy_Jaccard)
print("Accuracy of Cosine-K-means:",Accuracy_Cosine)

Tolerance for the Iteration  0 :  23941.31451612903
Tolerance for the Iteration  1 :  6264.812273534962
Tolerance for the Iteration  2 :  3526.9796048575568
Tolerance for the Iteration  3 :  2868.1528407071783
Tolerance for the Iteration  4 :  2583.195438664596
Tolerance for the Iteration  5 :  2367.617670607951
Tolerance for the Iteration  6 :  1617.458244111349
Tolerance for the Iteration  7 :  1346.1785811738916
Tolerance for the Iteration  8 :  1162.5874153569325
Tolerance for the Iteration  9 :  945.9941534624411
Tolerance for the Iteration  10 :  990.6466743041983
Tolerance for the Iteration  11 :  942.3846944678987
Tolerance for the Iteration  12 :  862.3541339731626
Tolerance for the Iteration  13 :  846.1567395325819
Tolerance for the Iteration  14 :  755.8401555341211
Tolerance for the Iteration  15 :  649.3220307711476
Tolerance for the Iteration  16 :  559.4559393522029
Tolerance for the Iteration  17 :  391.44332043705634
Tolerance for the Iteration  18 :  313.154841357398